In [38]:
!pip install torch
!pip install gensim
!pip install nltk
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install transformers

!pip install keras
!pip install faiss-gpu

!pip install tensorflow
import keras
import tensorflow
import faiss



import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
# from torch.optim import AdamW
from transformers import BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KDTree
import pandas as pd
import numpy as np
import json
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
if torch.cuda.is_available():
    torch.cuda.empty_cache()





Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [39]:
df1 = pd.read_csv('./df1.csv', nrows=10000)
df1['abstract'] = df1['abstract'].astype(str)
df1.head(10)

,abstract,astro-ph,astro-ph.CO,astro-ph.EP,astro-ph.GA,astro-ph.HE,astro-ph.IM,astro-ph.SR,cond-mat.dis-nn,cond-mat.mes-hall,...,q-fin.RM,q-fin.ST,q-fin.TR,quant-ph,stat.AP,stat.CO,stat.ME,stat.ML,stat.OT,stat.TH
0,many firms days opting specialize rather gener...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,paper analyses changes public opinion tracking...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,randomaccess networks ieee 80211 network diffe...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,mimo wireless channel offers rich ground quali...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,article presents new search algorithm nphard p...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,alice seeks informationtheoretically secure so...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,algorithms generate various combinatorial stru...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,describe fast method eliminate features variab...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,consider twouser gaussian multiple access chan...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,paper propose robust transceiver design kpair ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:

# 加载Word2Vec模型
model_path = "./GoogleNews-vectors-negative300.bin"
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

print(3)

3


In [42]:
# 确保这是你的nltk_data目录所在的路径
nltk_data_path = 'autodl-fs/nltk_data2'

# 添加到NLTK的数据路径
nltk.data.path.append(nltk_data_path)


nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)

print(4)


4


[nltk_data] Downloading package punkt to autodl-fs/nltk_data2...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to autodl-fs/nltk_data2...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to autodl-fs/nltk_data2...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:

# 分词和预处理函数
def tokenize_and_process(text):
    words = word_tokenize(text)
    words = [word for word in words if word.isalpha()]
    words = [word.lower() for word in words]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words


# 对DataFrame中的abstract列进行分词和预处理
df1['tokenized_abstract'] = df1['abstract'].apply(tokenize_and_process)

# 函数来将单词转换为Word2Vec向量，如果模型中没有该词，则使用UNK向量
def word_to_vec(word, model):
    return model[word] if word in model.key_to_index else model['UNK']

# 函数来将句子的分词列表转换为Word2Vec向量列表
def tokens_to_vectors(tokens, model):
    return [word_to_vec(token, model) for token in tokens]

# 应用函数将tokenized_abstract列的分词列表转换为Word2Vec向量列表
df1['word_vectors'] = df1['tokenized_abstract'].apply(lambda tokens: tokens_to_vectors(tokens, word2vec_model))


In [44]:


# 找出word_vectors中最长的长度
MAX_SEQUENCE_LENGTH = df1['word_vectors'].apply(len).max()
UNK_VECTOR = word2vec_model['UNK']  # 获取'UNK'的向量表示
# 使用这个长度作为填充或截断的基础
def pad_or_truncate_vectors(word_vectors, max_length, unk_vector):
    """
    如果word_vectors长度小于max_length，则用unk_vector填充；
    如果word_vectors长度大于max_length，则截断。
    """
    # 获取当前word vectors的长度
    sequence_length = len(word_vectors)
    
    # 如果当前长度小于最大长度，进行填充
    if sequence_length < max_length:
        padding = [unk_vector] * (max_length - sequence_length)
        word_vectors.extend(padding)
    # 如果当前长度大于最大长度，进行截断
    elif sequence_length > max_length:
        word_vectors = word_vectors[:max_length]
    
    return word_vectors

# 应用函数pad_or_truncate_vectors到每一行的word_vectors列
df1['padded_word_vectors'] = df1['word_vectors'].apply(
    lambda x: pad_or_truncate_vectors(x, MAX_SEQUENCE_LENGTH, UNK_VECTOR))

# 检查结果
df1['padded_word_vectors'].apply(len)  # 每个向量的长度都应该是MAX_SEQUENCE_LENGTH

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 准备特征：将padded_word_vectors的列表转换为NumPy数组
X = np.array(df1['padded_word_vectors'].tolist())

# 准备标签：获取所有标签列
label_columns = df1.columns.difference(['abstract', 'tokenized_abstract', 'word_vectors', 'padded_word_vectors'])
y = df1[label_columns].values

# 分割数据集为训练集和测试集，这里使用20%的数据作为测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 输出分割后的数据集维度，仅用于确认 
print(f'Training set shape: {X_train.shape, y_train.shape}')
print(f'Test set shape: {X_test.shape, y_test.shape}')


Training set shape: ((8000, 217, 300), (8000, 145))
Test set shape: ((2000, 217, 300), (2000, 145))


In [45]:
print(f'Training set shape: {X_train.shape, y_train.shape}')
print(f'Test set shape: {X_test.shape, y_test.shape}')


Training set shape: ((8000, 217, 300), (8000, 145))
Test set shape: ((2000, 217, 300), (2000, 145))


In [50]:
from keras.models import Model
from keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Layer
from keras import backend as K
import tensorflow as tf



def micro_f1(y_true, y_pred):
    # 预测值大于0.3的被认为是正类
    y_pred = K.cast(K.greater(y_pred, 0.3), K.floatx())
    
    # 计算真正例、假正例和假负例
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    
    # 计算精确度和召回率
    precision = K.sum(true_positives) / (K.sum(predicted_positives) + K.epsilon())
    recall = K.sum(true_positives) / (K.sum(possible_positives) + K.epsilon())
    
    # 计算micro-F1分数
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1


# 模型参数
max_sequence_length = 217  # 句子的最大长度
embedding_dim = 300  # 词嵌入的维度
num_labels = 145  # 标签的数量


# 模型输入和结构定义
input_ = Input(shape=(max_sequence_length, embedding_dim),name='input_')

# 可以通过交叉验证调整filters和kernel_size
conv = Conv1D(filters=128, kernel_size=3, activation='relu')(input_)
gmp = GlobalMaxPooling1D(name='gmp')(conv)
dropout = Dropout(0.5)(gmp)
# 可以通过交叉验证调整units
dense = Dense(128, activation='relu')(dropout)
output = Dense(num_labels, activation='sigmoid', name='output')(dense)


# 真实标签的输入
y_true_input = Input(shape=(num_labels,), name='y_true_input')

# 创建模型实例
model = Model(inputs=input_, outputs=output)
def contrastive_loss_function(y_true, features, tau_prime):
    # 计算标签相似度矩阵
    label_similarities = tf.matmul(y_true, y_true, transpose_b=True)
    
    # 动态系数归一化
    beta = label_similarities / tf.reduce_sum(label_similarities, axis=1, keepdims=True)
    
    # 特征标准化
    features_norm = tf.nn.l2_normalize(features, axis=1)
    
    # 计算特征间的成对欧式距离
    pairwise_distances = tf.norm(tf.expand_dims(features_norm, 1) - tf.expand_dims(features_norm, 0), axis=2)
    
    # 对比损失计算
    L_con = -beta * tf.math.reduce_logsumexp(-pairwise_distances / tau_prime, axis=1)
    
    # 将对比损失加和
    contrastive_loss = tf.reduce_sum(L_con)
    
    return contrastive_loss

# 自定义层以计算对比损失
class ContrastiveLossLayer(Layer):
    def __init__(self, gamma=0.5, tau_prime=10, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma      # 对比损失的权重参数
        self.tau_prime = tau_prime  # 温度参数

    def call(self, inputs):
        y_true, features = inputs
        # 计算对比损失，传入温度参数 tau_prime
        loss = contrastive_loss_function(y_true, features, self.tau_prime)
        # 应用权重 gamma
        self.add_loss(loss * self.gamma)
        # 返回与 features 形状相同的零张量
        return tf.zeros_like(features)


# 实例化对比损失层并设置gamma值
contrastive_loss_layer = ContrastiveLossLayer(gamma=0.5, name='contrastive_loss')

# 获取gmp层的输出用于对比损失计算
gmp_output = model.get_layer('gmp').output

# 调用对比损失层，并将真实标签和gmp层的输出传入
contrastive_loss_layer([y_true_input, gmp_output])
model = Model(inputs=[input_, y_true_input], outputs=[output,gmp])


# 编译模型时，只需要指定主输出的损失函数
model.compile(optimizer='adam',
              loss={'output': 'binary_crossentropy', 'gmp': None},
              metrics={'output': micro_f1})

# 训练模型时的输入：特征和真实标签
model.fit([X_train, y_train],
          {'output': y_train},
          batch_size=32,
          epochs=40,
          validation_split=0.1)





Epoch 1/40
225/225 [==============================] - 2s 5ms/step - loss: 0.0941 - output_loss: 0.0941 - output_micro_f1: 0.1240 - val_loss: 0.0565 - val_output_loss: 0.0565 - val_output_micro_f1: 0.2320
Epoch 2/40
225/225 [==============================] - 1s 4ms/step - loss: 0.0553 - output_loss: 0.0553 - output_micro_f1: 0.2821 - val_loss: 0.0499 - val_output_loss: 0.0499 - val_output_micro_f1: 0.3336
Epoch 3/40
225/225 [==============================] - 1s 4ms/step - loss: 0.0504 - output_loss: 0.0504 - output_micro_f1: 0.3714 - val_loss: 0.0466 - val_output_loss: 0.0466 - val_output_micro_f1: 0.3944
Epoch 4/40
225/225 [==============================] - 1s 4ms/step - loss: 0.0470 - output_loss: 0.0470 - output_micro_f1: 0.4258 - val_loss: 0.0431 - val_output_loss: 0.0431 - val_output_micro_f1: 0.4515
Epoch 5/40
225/225 [==============================] - 1s 4ms/step - loss: 0.0442 - output_loss: 0.0442 - output_micro_f1: 0.4627 - val_loss: 0.0415 - val_output_loss: 0.0415 - val_outp

In [51]:
# 在测试集上评估模型性能
evaluation_results = model.evaluate([X_test,y_test], y_test)

# evaluation_results[0] 是整体的损失值
# evaluation_results[1] 是主输出层的损失值
# evaluation_results[2] 是主输出层的micro-F1分数

# 打印测试集上的性能
print(f'Test loss (overall): {evaluation_results[0]}')
print(f'Test loss (main output): {evaluation_results[1]}')
print(f'Test micro-F1 score (main output): {evaluation_results[2]}')





63/63 [==============================] - 0s 3ms/step - loss: 0.0381 - output_loss: 0.0381 - output_micro_f1: 0.5736
Test loss (overall): 0.038086581975221634
Test loss (main output): 0.038086581975221634
Test micro-F1 score (main output): 0.5736383199691772


In [52]:
# Extract features from the training set using the trained model
# This will be used for constructing the datastore for k-NN
train_features = model.predict([X_train,y_train])[1]
from sklearn.neighbors import NearestNeighbors  # 这里导入NearestNeighbors

# Construct the datastore for k-NN using the extracted features and the known labels
datastore = NearestNeighbors(n_neighbors=5).fit(train_features)

# Define k-NN inference function
def knn_inference(model, datastore, x_test, k=3, temperature=0.5, lambda_factor=0.8):
    cnn_test_predictions, test_features = model.predict(x_test)
    
    knn_test_predictions = np.zeros(cnn_test_predictions.shape)
    
    for i, feature in enumerate(test_features):
        distances, indices = datastore.kneighbors([feature], n_neighbors=k)
        weights = np.exp(-np.array(distances) / temperature)
        weights = weights / np.sum(weights)
        
        for idx, w in zip(indices[0], weights[0]):
            knn_test_predictions[i] += w * y_train[idx]
    
    knn_test_predictions /= np.max(knn_test_predictions, axis=1, keepdims=True)
    
    # Combine CNN model's output with k-NN's prediction to get the final prediction
    final_predictions = lambda_factor * cnn_test_predictions + (1 - lambda_factor) * knn_test_predictions
    return final_predictions

# Perform inference with k-NN on the test set
final_predictions = knn_inference(model, datastore, [X_test,y_test])

63/63 [==============================] - 0s 2ms/step


In [53]:
import numpy as np
from sklearn.metrics import f1_score, mean_squared_error

# 定义阈值
threshold = 0.3

# 将概率转换为二进制标签
y_pred_binary = (final_predictions >= threshold).astype(int)

# 计算micro-F1分数
micro_f1_score = f1_score(y_test, y_pred_binary, average='micro')

# 打印micro-F1分数
print(f'Micro-F1 score for final predictions: {micro_f1_score}')

# 计算测试误差（均方误差）
test_error = mean_squared_error(y_test, final_predictions)

# 打印测试误差
print(f'Test mean squared error: {test_error}')


Micro-F1 score for final predictions: 0.5771780853786205
Test mean squared error: 0.008857158934618355
